# Large-Scale Stochastic Variational GP Regression (CUDA) (w/ SVGP)

## Overview

In this notebook, we'll give an overview of how to use Deep Kernel Learning with SVGP stochastic variational regression to rapidly train using minibatches on the `3droad` UCI dataset with hundreds of thousands of training examples. 

In contrast to the SVDKL_Regression_GridInterp_CUDA notebook, we'll be using SVGP (https://arxiv.org/pdf/1411.2005.pdf) here to learn the inducing point locations. Our implementation of SVGP is modified to be efficient with the inference techniques used in GPyTorch.

In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

# Make plots inline
%matplotlib inline

## Loading Data

For this example notebook, we'll be using the `song` UCI dataset used in the paper. Running the next cell downloads a copy of the dataset that has already been scaled and normalized appropriately. For this notebook, we'll simply be splitting the data using the first 80% of the data as training and the last 20% as testing.

**Note**: Running the next cell will attempt to download a **~136 MB** file to the current directory.

In [2]:
import urllib.request
import os.path
from scipy.io import loadmat
from math import floor

if not os.path.isfile('3droad.mat'):
    print('Downloading \'3droad\' UCI dataset...')
    urllib.request.urlretrieve('https://www.dropbox.com/s/f6ow1i59oqx05pl/3droad.mat?dl=1', '3droad.mat')
    
data = torch.Tensor(loadmat('3droad.mat')['data'])
X = data[:, :-1]
X = X - X.min(0)[0]
X = 2 * (X / X.max(0)[0]) - 1
y = data[:, -1]

# Use the first 80% of the data for training, and the last 20% for testing.
train_n = int(floor(0.8*len(X)))

train_x = X[:train_n, :].contiguous().cuda()
train_y = y[:train_n].contiguous().cuda()

test_x = X[train_n:, :].contiguous().cuda()
test_y = y[train_n:].contiguous().cuda()

## Creating a DataLoader

The next step is to create a torch `DataLoader` that will handle getting us random minibatches of data. This involves using the standard `TensorDataset` and `DataLoader` modules provided by PyTorch.

In this notebook we'll be using a fairly large batch size of 1024 just to make optimization run faster, but you could of course change this as you so choose.

In [3]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

## Defining the DKL Feature Extractor

Next, we define the neural network feature extractor used to define the deep kernel. In this case, we use a fully connected network with the architecture `d -> 1000 -> 500 -> 50 -> 2`, as described in the original DKL paper. All of the code below uses standard PyTorch implementations of neural network layers.

In [4]:
data_dim = train_x.size(-1)

class LargeFeatureExtractor(torch.nn.Sequential):           
    def __init__(self):                                      
        super(LargeFeatureExtractor, self).__init__()        
        self.add_module('linear1', torch.nn.Linear(data_dim, 1000))
        self.add_module('bn1', torch.nn.BatchNorm1d(1000))
        self.add_module('relu1', torch.nn.ReLU())
        self.add_module('linear2', torch.nn.Linear(1000, 1000))
        self.add_module('bn2', torch.nn.BatchNorm1d(1000))
        self.add_module('relu2', torch.nn.ReLU())                       
        self.add_module('linear3', torch.nn.Linear(1000, 500))
        self.add_module('bn3', torch.nn.BatchNorm1d(500))
        self.add_module('relu3', torch.nn.ReLU())                  
        self.add_module('linear4', torch.nn.Linear(500, 50))       
        self.add_module('bn4', torch.nn.BatchNorm1d(50))
        self.add_module('relu4', torch.nn.ReLU())                  
        self.add_module('linear5', torch.nn.Linear(50, 2))         
                                                             
feature_extractor = LargeFeatureExtractor().cuda()
# num_features is the number of final features extracted by the neural network, in this case 2.
num_features = 2

## Defining the GP Regression Layer

We now define the GP regression module that, intuitvely, will act as the final "layer" of our neural network. In this case, because we are doing variational inference and *not* exact inference, we will be using an `AbstractVariationalGP`. In this example, because we will be learning the inducing point locations, we'll be using a base `VariationalStrategy` with `learn_inducing_locations=True`.

Because the feature extractor we defined above extracts two features, we'll need to define our grid bounds over two dimensions.

In [5]:
from gpytorch.models import AbstractVariationalGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy

softplus = torch.functional.F.softplus

class GPRegressionLayer(AbstractVariationalGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(GPRegressionLayer, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(param_transform=softplus), param_transform=softplus
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

## Defining the DKL Model

With the feature extractor and GP regression layer defined, we can now define our full model. To do this, we simply create a module whose `forward()` method passes the data first through the feature extractor, and then through the GP regression layer.

The only other interesting feature of the model below is that we use a helper function, `scale_to_bounds`, to ensure that the features extracted by the neural network fit within the grid bounds used for SKI.

In [6]:
class DKLModel(gpytorch.Module):
    def __init__(self, inducing_points, feature_extractor, num_features, grid_bounds=(-1., 1.)):
        super(DKLModel, self).__init__()
        self.feature_extractor = feature_extractor
        self.gp_layer = GPRegressionLayer(inducing_points)
        self.grid_bounds = grid_bounds
        self.num_features = num_features

    def forward(self, x):
        features = self.feature_extractor(x)
        features = gpytorch.utils.grid.scale_to_bounds(features, self.grid_bounds[0], self.grid_bounds[1])
        res = self.gp_layer(features)
        return res
inducing_points = gpytorch.utils.grid.scale_to_bounds(feature_extractor(train_x[:500, :]), -1, 1)
model = DKLModel(inducing_points=inducing_points, feature_extractor=feature_extractor, num_features=num_features).cuda()
likelihood = gpytorch.likelihoods.GaussianLikelihood(param_transform=softplus).cuda()

## Training the Model

The cell below trains the DKL model above, learning both the hyperparameters of the Gaussian process **and** the parameters of the neural network in an end-to-end fashion using Type-II MLE.

Unlike when using the exact GP marginal log likelihood, performing variational inference allows us to make use of stochastic optimization techniques. For this example, we'll do one epoch of training. Given the small size of the neural network relative to the size of the dataset, this should be sufficient to achieve comparable accuracy to what was observed in the DKL paper.

The optimization loop differs from the one seen in our more simple tutorials in that it involves looping over both a number of training iterations (epochs) *and* minibatches of the data. However, the basic process is the same: for each minibatch, we forward through the model, compute the loss (the `VariationalMarginalLogLikelihood` or ELBO), call backwards, and do a step of optimization.

In [7]:
model.train()
likelihood.train()

# We'll do 1 epochs of training in this tutorial
num_epochs = 6

# We use SGD here, rather than Adam. Emperically, we find that SGD is better for variational regression
optimizer = torch.optim.Adam([
    {'params': model.feature_extractor.parameters(), 'weight_decay': 1e-3},
    {'params': model.gp_layer.parameters()},
    {'params': likelihood.parameters()},
], lr=0.1)

scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 5], gamma=0.1)

# Our loss object. We're using the VariationalELBO, which essentially just computes the ELBO
mll = gpytorch.mlls.VariationalELBO(likelihood, model.gp_layer, num_data=train_y.size(0), combine_terms=False)

for i in range(num_epochs):
    scheduler.step()
    # Within each iteration, we will go over each minibatch of data
    for minibatch_i, (x_batch, y_batch) in enumerate(train_loader):        
        optimizer.zero_grad()
        
        # Because the grid is relatively small, we turn off the Toeplitz matrix multiplication and just perform them directly
        # We find this to be more efficient when the grid is very small.
        with gpytorch.settings.use_toeplitz(False):
            output = model(x_batch)
            log_lik, kl_div, log_prior = mll(output, y_batch)
            loss = -(log_lik - kl_div + log_prior)
            print('Epoch %d [%d/%d] - Loss: %.3f [%.3f, %.3f, %.3f]' % (i + 1, minibatch_i, len(train_loader), loss.item(), log_lik.item(), kl_div.item(), log_prior.item()))

        # The actual optimization step
        loss.backward()
        optimizer.step()

Epoch 1 [0/340] - Loss: 231.386 [-231.386, 0.000, 0.000]
Epoch 1 [1/340] - Loss: 224.000 [-223.660, 0.340, 0.000]
Epoch 1 [2/340] - Loss: 215.339 [-214.659, 0.680, 0.000]
Epoch 1 [3/340] - Loss: 238.089 [-237.241, 0.848, 0.000]
Epoch 1 [4/340] - Loss: 219.715 [-218.678, 1.037, 0.000]
Epoch 1 [5/340] - Loss: 200.558 [-199.142, 1.416, 0.000]
Epoch 1 [6/340] - Loss: 177.126 [-175.359, 1.767, 0.000]
Epoch 1 [7/340] - Loss: 168.512 [-166.541, 1.971, 0.000]
Epoch 1 [8/340] - Loss: 154.651 [-152.548, 2.103, 0.000]
Epoch 1 [9/340] - Loss: 125.872 [-123.504, 2.368, 0.000]
Epoch 1 [10/340] - Loss: 137.950 [-135.491, 2.460, 0.000]
Epoch 1 [11/340] - Loss: 127.375 [-124.782, 2.593, 0.000]
Epoch 1 [12/340] - Loss: 123.587 [-120.999, 2.588, 0.000]
Epoch 1 [13/340] - Loss: 119.784 [-117.043, 2.741, 0.000]
Epoch 1 [14/340] - Loss: 116.733 [-113.707, 3.026, 0.000]
Epoch 1 [15/340] - Loss: 110.453 [-107.430, 3.023, 0.000]
Epoch 1 [16/340] - Loss: 104.404 [-101.405, 2.999, 0.000]
Epoch 1 [17/340] - Loss:

Epoch 1 [148/340] - Loss: 26.419 [-26.223, 0.196, 0.000]
Epoch 1 [149/340] - Loss: 29.123 [-28.923, 0.199, 0.000]
Epoch 1 [150/340] - Loss: 29.041 [-28.836, 0.205, 0.000]
Epoch 1 [151/340] - Loss: 27.033 [-26.834, 0.199, 0.000]
Epoch 1 [152/340] - Loss: 27.389 [-27.191, 0.198, 0.000]
Epoch 1 [153/340] - Loss: 30.345 [-30.148, 0.198, 0.000]
Epoch 1 [154/340] - Loss: 29.007 [-28.806, 0.200, 0.000]
Epoch 1 [155/340] - Loss: 25.998 [-25.798, 0.200, 0.000]
Epoch 1 [156/340] - Loss: 26.288 [-26.086, 0.201, 0.000]
Epoch 1 [157/340] - Loss: 25.736 [-25.532, 0.204, 0.000]
Epoch 1 [158/340] - Loss: 26.964 [-26.755, 0.210, 0.000]
Epoch 1 [159/340] - Loss: 24.904 [-24.691, 0.213, 0.000]
Epoch 1 [160/340] - Loss: 27.456 [-27.247, 0.209, 0.000]
Epoch 1 [161/340] - Loss: 27.573 [-27.366, 0.207, 0.000]
Epoch 1 [162/340] - Loss: 24.242 [-24.043, 0.200, 0.000]
Epoch 1 [163/340] - Loss: 25.195 [-24.996, 0.199, 0.000]
Epoch 1 [164/340] - Loss: 24.416 [-24.219, 0.197, 0.000]
Epoch 1 [165/340] - Loss: 24.05

Epoch 1 [293/340] - Loss: 18.354 [-18.277, 0.077, 0.000]
Epoch 1 [294/340] - Loss: 18.551 [-18.474, 0.077, 0.000]
Epoch 1 [295/340] - Loss: 18.300 [-18.223, 0.077, 0.000]
Epoch 1 [296/340] - Loss: 18.642 [-18.566, 0.076, 0.000]
Epoch 1 [297/340] - Loss: 18.463 [-18.387, 0.076, 0.000]
Epoch 1 [298/340] - Loss: 18.188 [-18.113, 0.075, 0.000]
Epoch 1 [299/340] - Loss: 18.245 [-18.171, 0.074, 0.000]
Epoch 1 [300/340] - Loss: 18.958 [-18.884, 0.073, 0.000]
Epoch 1 [301/340] - Loss: 18.261 [-18.188, 0.073, 0.000]
Epoch 1 [302/340] - Loss: 18.895 [-18.821, 0.073, 0.000]
Epoch 1 [303/340] - Loss: 17.181 [-17.108, 0.073, 0.000]
Epoch 1 [304/340] - Loss: 15.932 [-15.857, 0.074, 0.000]
Epoch 1 [305/340] - Loss: 17.802 [-17.729, 0.074, 0.000]
Epoch 1 [306/340] - Loss: 17.160 [-17.086, 0.074, 0.000]
Epoch 1 [307/340] - Loss: 16.328 [-16.254, 0.074, 0.000]
Epoch 1 [308/340] - Loss: 19.513 [-19.438, 0.074, 0.000]
Epoch 1 [309/340] - Loss: 17.287 [-17.213, 0.074, 0.000]
Epoch 1 [310/340] - Loss: 19.24

Epoch 2 [99/340] - Loss: 20.669 [-20.561, 0.109, 0.000]
Epoch 2 [100/340] - Loss: 19.873 [-19.755, 0.117, 0.000]
Epoch 2 [101/340] - Loss: 19.939 [-19.811, 0.128, 0.000]
Epoch 2 [102/340] - Loss: 19.191 [-19.062, 0.128, 0.000]
Epoch 2 [103/340] - Loss: 19.372 [-19.250, 0.123, 0.000]
Epoch 2 [104/340] - Loss: 19.072 [-18.948, 0.124, 0.000]
Epoch 2 [105/340] - Loss: 20.560 [-20.438, 0.122, 0.000]
Epoch 2 [106/340] - Loss: 19.183 [-19.059, 0.123, 0.000]
Epoch 2 [107/340] - Loss: 18.336 [-18.231, 0.105, 0.000]
Epoch 2 [108/340] - Loss: 19.943 [-19.843, 0.100, 0.000]
Epoch 2 [109/340] - Loss: 21.090 [-20.994, 0.096, 0.000]
Epoch 2 [110/340] - Loss: 19.748 [-19.654, 0.094, 0.000]
Epoch 2 [111/340] - Loss: 19.492 [-19.397, 0.095, 0.000]
Epoch 2 [112/340] - Loss: 21.237 [-21.142, 0.096, 0.000]
Epoch 2 [113/340] - Loss: 20.960 [-20.862, 0.098, 0.000]
Epoch 2 [114/340] - Loss: 22.070 [-21.972, 0.098, 0.000]
Epoch 2 [115/340] - Loss: 19.819 [-19.719, 0.100, 0.000]
Epoch 2 [116/340] - Loss: 20.464

Epoch 2 [243/340] - Loss: 15.820 [-15.754, 0.066, 0.000]
Epoch 2 [244/340] - Loss: 16.167 [-16.103, 0.064, 0.000]
Epoch 2 [245/340] - Loss: 15.611 [-15.549, 0.061, 0.000]
Epoch 2 [246/340] - Loss: 16.774 [-16.714, 0.060, 0.000]
Epoch 2 [247/340] - Loss: 15.938 [-15.880, 0.058, 0.000]
Epoch 2 [248/340] - Loss: 15.428 [-15.372, 0.056, 0.000]
Epoch 2 [249/340] - Loss: 15.548 [-15.493, 0.055, 0.000]
Epoch 2 [250/340] - Loss: 14.927 [-14.873, 0.054, 0.000]
Epoch 2 [251/340] - Loss: 17.893 [-17.840, 0.053, 0.000]
Epoch 2 [252/340] - Loss: 16.159 [-16.106, 0.053, 0.000]
Epoch 2 [253/340] - Loss: 14.125 [-14.072, 0.053, 0.000]
Epoch 2 [254/340] - Loss: 14.751 [-14.699, 0.053, 0.000]
Epoch 2 [255/340] - Loss: 14.690 [-14.638, 0.052, 0.000]
Epoch 2 [256/340] - Loss: 16.026 [-15.974, 0.052, 0.000]
Epoch 2 [257/340] - Loss: 14.982 [-14.930, 0.052, 0.000]
Epoch 2 [258/340] - Loss: 16.640 [-16.587, 0.053, 0.000]
Epoch 2 [259/340] - Loss: 14.597 [-14.544, 0.053, 0.000]
Epoch 2 [260/340] - Loss: 15.43

...

Epoch 4 [291/340] - Loss: 12.952 [-12.924, 0.027, 0.000]
Epoch 4 [292/340] - Loss: 12.806 [-12.779, 0.027, 0.000]
Epoch 4 [293/340] - Loss: 12.306 [-12.279, 0.027, 0.000]
Epoch 4 [294/340] - Loss: 13.280 [-13.253, 0.027, 0.000]
Epoch 4 [295/340] - Loss: 13.592 [-13.565, 0.027, 0.000]
Epoch 4 [296/340] - Loss: 13.043 [-13.016, 0.027, 0.000]
Epoch 4 [297/340] - Loss: 13.619 [-13.592, 0.027, 0.000]
Epoch 4 [298/340] - Loss: 13.290 [-13.263, 0.027, 0.000]
Epoch 4 [299/340] - Loss: 12.552 [-12.524, 0.027, 0.000]
Epoch 4 [300/340] - Loss: 12.227 [-12.200, 0.027, 0.000]
Epoch 4 [301/340] - Loss: 12.515 [-12.488, 0.027, 0.000]
Epoch 4 [302/340] - Loss: 13.322 [-13.294, 0.027, 0.000]
Epoch 4 [303/340] - Loss: 13.307 [-13.280, 0.027, 0.000]
Epoch 4 [304/340] - Loss: 13.677 [-13.650, 0.027, 0.000]
Epoch 4 [305/340] - Loss: 13.219 [-13.192, 0.027, 0.000]
Epoch 4 [306/340] - Loss: 13.188 [-13.161, 0.027, 0.000]
Epoch 4 [307/340] - Loss: 14.022 [-13.995, 0.027, 0.000]
Epoch 4 [308/340] - Loss: 12.40

Epoch 5 [100/340] - Loss: 13.365 [-13.337, 0.028, 0.000]
Epoch 5 [101/340] - Loss: 12.903 [-12.874, 0.028, 0.000]
Epoch 5 [102/340] - Loss: 12.849 [-12.821, 0.028, 0.000]
Epoch 5 [103/340] - Loss: 12.004 [-11.976, 0.028, 0.000]
Epoch 5 [104/340] - Loss: 12.388 [-12.360, 0.028, 0.000]
Epoch 5 [105/340] - Loss: 13.585 [-13.557, 0.028, 0.000]
Epoch 5 [106/340] - Loss: 12.205 [-12.177, 0.028, 0.000]
Epoch 5 [107/340] - Loss: 11.884 [-11.857, 0.028, 0.000]
Epoch 5 [108/340] - Loss: 13.347 [-13.319, 0.028, 0.000]
Epoch 5 [109/340] - Loss: 13.563 [-13.535, 0.028, 0.000]
Epoch 5 [110/340] - Loss: 11.751 [-11.723, 0.028, 0.000]
Epoch 5 [111/340] - Loss: 12.299 [-12.271, 0.028, 0.000]
Epoch 5 [112/340] - Loss: 12.474 [-12.446, 0.028, 0.000]
Epoch 5 [113/340] - Loss: 11.942 [-11.914, 0.028, 0.000]
Epoch 5 [114/340] - Loss: 13.135 [-13.107, 0.028, 0.000]
Epoch 5 [115/340] - Loss: 12.432 [-12.404, 0.028, 0.000]
Epoch 5 [116/340] - Loss: 13.028 [-13.000, 0.028, 0.000]
Epoch 5 [117/340] - Loss: 14.03

Epoch 5 [246/340] - Loss: 11.755 [-11.729, 0.026, 0.000]
Epoch 5 [247/340] - Loss: 14.237 [-14.210, 0.027, 0.000]
Epoch 5 [248/340] - Loss: 12.758 [-12.731, 0.027, 0.000]
Epoch 5 [249/340] - Loss: 13.292 [-13.265, 0.027, 0.000]
Epoch 5 [250/340] - Loss: 12.635 [-12.607, 0.027, 0.000]
Epoch 5 [251/340] - Loss: 13.644 [-13.617, 0.027, 0.000]
Epoch 5 [252/340] - Loss: 13.271 [-13.244, 0.027, 0.000]
Epoch 5 [253/340] - Loss: 13.399 [-13.371, 0.027, 0.000]
Epoch 5 [254/340] - Loss: 12.840 [-12.812, 0.028, 0.000]
Epoch 5 [255/340] - Loss: 13.903 [-13.876, 0.028, 0.000]
Epoch 5 [256/340] - Loss: 12.903 [-12.875, 0.027, 0.000]
Epoch 5 [257/340] - Loss: 13.768 [-13.741, 0.027, 0.000]
Epoch 5 [258/340] - Loss: 12.376 [-12.349, 0.027, 0.000]
Epoch 5 [259/340] - Loss: 12.652 [-12.625, 0.027, 0.000]
Epoch 5 [260/340] - Loss: 12.615 [-12.588, 0.027, 0.000]
Epoch 5 [261/340] - Loss: 12.891 [-12.864, 0.027, 0.000]
Epoch 5 [262/340] - Loss: 12.764 [-12.737, 0.027, 0.000]
Epoch 5 [263/340] - Loss: 12.77

Epoch 6 [51/340] - Loss: 13.185 [-13.159, 0.026, 0.000]
Epoch 6 [52/340] - Loss: 12.518 [-12.491, 0.026, 0.000]
Epoch 6 [53/340] - Loss: 12.571 [-12.545, 0.026, 0.000]
Epoch 6 [54/340] - Loss: 12.258 [-12.232, 0.026, 0.000]
Epoch 6 [55/340] - Loss: 12.716 [-12.691, 0.026, 0.000]
Epoch 6 [56/340] - Loss: 12.322 [-12.296, 0.026, 0.000]
Epoch 6 [57/340] - Loss: 12.851 [-12.825, 0.026, 0.000]
Epoch 6 [58/340] - Loss: 12.403 [-12.378, 0.026, 0.000]
Epoch 6 [59/340] - Loss: 12.406 [-12.380, 0.026, 0.000]
Epoch 6 [60/340] - Loss: 12.402 [-12.376, 0.026, 0.000]
Epoch 6 [61/340] - Loss: 12.913 [-12.887, 0.026, 0.000]
Epoch 6 [62/340] - Loss: 12.477 [-12.451, 0.026, 0.000]
Epoch 6 [63/340] - Loss: 12.276 [-12.250, 0.026, 0.000]
Epoch 6 [64/340] - Loss: 12.153 [-12.127, 0.026, 0.000]
Epoch 6 [65/340] - Loss: 12.892 [-12.866, 0.026, 0.000]
Epoch 6 [66/340] - Loss: 12.269 [-12.243, 0.026, 0.000]
Epoch 6 [67/340] - Loss: 13.347 [-13.322, 0.026, 0.000]
Epoch 6 [68/340] - Loss: 12.587 [-12.561, 0.026,

Epoch 6 [198/340] - Loss: 12.351 [-12.325, 0.026, 0.000]
Epoch 6 [199/340] - Loss: 12.576 [-12.550, 0.026, 0.000]
Epoch 6 [200/340] - Loss: 13.028 [-13.002, 0.026, 0.000]
Epoch 6 [201/340] - Loss: 12.523 [-12.497, 0.026, 0.000]
Epoch 6 [202/340] - Loss: 13.621 [-13.595, 0.026, 0.000]
Epoch 6 [203/340] - Loss: 12.466 [-12.440, 0.026, 0.000]
Epoch 6 [204/340] - Loss: 13.231 [-13.205, 0.026, 0.000]
Epoch 6 [205/340] - Loss: 12.857 [-12.831, 0.026, 0.000]
Epoch 6 [206/340] - Loss: 12.429 [-12.403, 0.026, 0.000]
Epoch 6 [207/340] - Loss: 12.896 [-12.870, 0.026, 0.000]
Epoch 6 [208/340] - Loss: 12.450 [-12.423, 0.026, 0.000]
Epoch 6 [209/340] - Loss: 12.141 [-12.115, 0.026, 0.000]
Epoch 6 [210/340] - Loss: 12.646 [-12.620, 0.026, 0.000]
Epoch 6 [211/340] - Loss: 12.611 [-12.585, 0.026, 0.000]
Epoch 6 [212/340] - Loss: 12.229 [-12.203, 0.026, 0.000]
Epoch 6 [213/340] - Loss: 12.090 [-12.063, 0.026, 0.000]
Epoch 6 [214/340] - Loss: 11.643 [-11.617, 0.026, 0.000]
Epoch 6 [215/340] - Loss: 13.33

## Making Predictions

The next cell gets the predictive covariance for the test set (and also technically gets the predictive mean, stored in `preds.mean()`) using the standard SKI testing code, with no acceleration or precomputation. Because the test set is substantially smaller than the training set, we don't need to make predictions in mini batches here, although our other tutorials demonstrate how to do this (for example, see the CIFAR tutorial).

In [8]:
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.use_toeplitz(False):
    preds = model(test_x)

In [9]:
print('Test MAE: {}'.format(torch.mean(torch.abs(preds.mean - test_y))))

Test MAE: 8.008056640625
